In [1]:
!nvcc -V
!echo
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git
!pip install -r /content/Swin-Transformer-Object-Detection/requirements.txt
!pip install openmim
!mim install mmdet

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

In [2]:
%cd Swin-Transformer-Object-Detection/

/content/Swin-Transformer-Object-Detection


In [ ]:
!unzip '/content/drive/MyDrive/input/ocr_ancient.zip' -d /content/data

In [7]:
# 목판본_043_언해류_대학언해_01
# 목판본_043_언해류_대학언해1_01
# 필사본_104_문학류_대명영렬전2_01
# 필사본_104_문학류_대병영렬전2_01
# 필사본_105_문학류_대명영렬전3_01
# 필사본_105_문학류_대병영렬전3_01
import os
from glob import glob
json_list = glob(os.path.join('/content/data/이미지데이터','*')) # derectory
for i in json_list:
    tmp_list = glob(os.path.join(i,'*'))
    for j in tmp_list: # image파일

        if j.split('/')[-1][:-8] != i.split('/')[-1]:
            dir_name = i.split('/')[-1]
            img_number = j.split('/')[-1][-8:]
            os.rename(j, '/content/data/이미지데이터/'+dir_name+'/'+dir_name+img_number)

In [3]:
import sys
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, set_random_seed, init_detector
from mmdet.datasets import build_dataloader, build_dataset, replace_ImageToTensor
from mmdet.utils import collect_env, get_root_logger

import torch
import time
import json

from pprint import pprint
import os

In [4]:
classes = (
    "word",
)
CUR_PATH = os.getcwd()
CFG_PATH = "/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/cascade_rcnn/cascade_rcnn_swin_tiny_fpn_1x_coco.py"
PREFIX = "/content/data/이미지데이터"
WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir"
# CHK_PATH = "/content/drive/MyDrive/Colab Notebooks/lesion_obd/cascade_mask_rcnn_swin_small_patch4_window7.pth"
# config file 들고오기
cfg = Config.fromfile(CFG_PATH)

In [16]:
for i in cfg:
    print(i, '*'*40)
    print(cfg[i])

model ****************************************
{'type': 'CascadeRCNN', 'pretrained': None, 'backbone': {'type': 'SwinTransformer', 'embed_dim': 96, 'depths': [2, 2, 6, 2], 'num_heads': [3, 6, 12, 24], 'window_size': 7, 'mlp_ratio': 4.0, 'qkv_bias': True, 'qk_scale': None, 'drop_rate': 0.0, 'attn_drop_rate': 0.0, 'drop_path_rate': 0.2, 'ape': False, 'patch_norm': True, 'out_indices': (0, 1, 2, 3), 'use_checkpoint': False}, 'neck': {'type': 'FPN', 'in_channels': [96, 192, 384, 768], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.7, 1.0, 1.3], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'SmoothL1Loss', 'beta': 0.1111111111111111, 'loss_weigh

In [6]:

cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = '/content/drive/MyDrive/input/train.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = '/content/drive/MyDrive/input/valid.json'

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 4

cfg.seed = 42
cfg.gpu_ids = [0]

distributed = False

cfg.work_dir = WORK_DIR
cfg.runner.max_epochs = 20
cfg.rtotal_epochs = 20
cfg.optimizer = dict(type='Adam', lr=0.0001, weight_decay=0.0001)

cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=1000, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

cfg.log_config.interval = 600
cfg.checkpoint_config.interval = 1
cfg.log_config = {'hooks': [{'type': 'TextLoggerHook'}], 'interval': 600}
# cfg.model.pretrained = None

model = build_detector(cfg.model)

datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

loading annotations into memory...
Done (t=24.23s)
creating index...
index created!


In [7]:
train_detector(model, datasets[0], cfg, distributed=distributed, validate=True)

loading annotations into memory...
Done (t=1.80s)
creating index...


2021-12-08 02:38:42,294 - mmdet - INFO - load checkpoint from local path: /content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/pth/swin_tiny_patch4_window7_224.pth


index created!


2021-12-08 02:38:42,707 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: model

missing keys in source state_dict: backbone.patch_embed.proj.weight, backbone.patch_embed.proj.bias, backbone.patch_embed.norm.weight, backbone.patch_embed.norm.bias, backbone.layers.0.blocks.0.norm1.weight, backbone.layers.0.blocks.0.norm1.bias, backbone.layers.0.blocks.0.attn.relative_position_bias_table, backbone.layers.0.blocks.0.attn.relative_position_index, backbone.layers.0.blocks.0.attn.qkv.weight, backbone.layers.0.blocks.0.attn.qkv.bias, backbone.layers.0.blocks.0.attn.proj.weight, backbone.layers.0.blocks.0.attn.proj.bias, backbone.layers.0.blocks.0.norm2.weight, backbone.layers.0.blocks.0.norm2.bias, backbone.layers.0.blocks.0.mlp.fc1.weight, backbone.layers.0.blocks.0.mlp.fc1.bias, backbone.layers.0.blocks.0.mlp.fc2.weight, backbone.layers.0.blocks.0.mlp.fc2.bias, backbone.layers.0.blocks.1.norm1.weight, backbone.layers.0.blocks.1.no

[                                ] 3/1604, 2.1 task/s, elapsed: 1s, ETA:   779s

...

Loading and preparing results...
DONE (t=0.38s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=609.95s).
Accumulating evaluation results...


2021-12-08 10:32:29,787 - mmdet - INFO - Epoch(val) [6][3590]	bbox_mAP: 0.0660, bbox_mAP_50: 0.1680, bbox_mAP_75: 0.1140, bbox_mAP_s: 0.0220, bbox_mAP_m: 0.1920, bbox_mAP_l: 0.6020, bbox_mAP_copypaste: 0.066 0.168 0.114 0.022 0.192 0.602


DONE (t=3.06s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.066
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.168
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.114
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.022
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.192
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.602
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.068
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.108
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.108
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.019
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.221
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.670


2021-12-08 10:44:00,617 - mmdet - INFO - Epoch [7][600/3590]	lr: 7.834e-05, eta: 12:50:45, time: 1.151, data_time: 0.016, memory: 12374, loss_rpn_cls: 0.0152, loss_rpn_bbox: 0.0154, s0.loss_cls: 0.0044, s0.acc: 99.8497, s0.loss_bbox: 0.0155, s1.loss_cls: 0.0025, s1.acc: 99.8304, s1.loss_bbox: 0.0143, s2.loss_cls: 0.0016, s2.acc: 99.7866, s2.loss_bbox: 0.0126, loss: 0.0816
2021-12-08 10:55:07,853 - mmdet - INFO - Epoch [7][1200/3590]	lr: 7.725e-05, eta: 12:45:20, time: 1.112, data_time: 0.009, memory: 12374, loss_rpn_cls: 0.0149, loss_rpn_bbox: 0.0150, s0.loss_cls: 0.0053, s0.acc: 99.8191, s0.loss_bbox: 0.0154, s1.loss_cls: 0.0031, s1.acc: 99.7940, s1.loss_bbox: 0.0144, s2.loss_cls: 0.0017, s2.acc: 99.7748, s2.loss_bbox: 0.0126, loss: 0.0825
2021-12-08 11:06:07,809 - mmdet - INFO - Epoch [7][1800/3590]	lr: 7.614e-05, eta: 12:39:23, time: 1.100, data_time: 0.009, memory: 12374, loss_rpn_cls: 0.0129, loss_rpn_bbox: 0.0136, s0.loss_cls: 0.0039, s0.acc: 99.8698, s0.loss_bbox: 0.0140, s1.los

KeyboardInterrupt: ignored